In [1]:
from matplotlib import pyplot as plt
import numpy as np

plt.style.use('ggplot')

In [2]:
# Analysis of 3D rotation fitting. A computational method of hip joint center

def TransformationMatrix(bodyVecs, globalVecs, weights):
    n = bodyVecs.shape[1]
    Pmean = np.mean(bodyVecs, axis=1)
    Qmean = np.mean(globalVecs, axis=1)
    
    P = bodyVecs - np.tile(Pmean.reshape(3,-1), (1,n))
    Q = globalVecs - np.tile(Qmean.reshape(3,-1), (1,n))
    
    K = np.zeros((3, 3))
    for i in range(n):
        K += weights[i] * np.matmul(Q[:,i].reshape(3,1), P[:,i].reshape(1,3))
    
    u,s,vh = np.linalg.svd(K)
    
    vin = np.identity(3)
    vin[2,2] = np.linalg.det(np.matmul(u,vh))
    
    R = np.matmul(u, np.matmul(vin, vh))
    d = Qmean-np.dot(R,Pmean)
    
    transformMat = np.identity(4)
    transformMat[0:3, 0:3] = R
    transformMat[0:3, 3] = d
    
    return transformMat

In [3]:
class fileReader:
    def __init__(self, fileName, columnLabelsLine=5, numHeaderLines=8, delimiter=","):
        self.Data = np.genfromtxt(fileName, delimiter=delimiter, skip_header=numHeaderLines)
        self.NumFrames = self.Data.shape[0]
        #self.NumFrames = 20000
        with open(fileName) as infile:
            content = infile.readlines()
            row = content[columnLabelsLine]
        self.dataIdx = {label:i for i,label in enumerate(row.split(delimiter)) if label != ""}
    
    def getDataForMarkers(self, nLine, markers):           
        result = np.zeros((3, len(markers)))
        for col, marker in enumerate(markers):
            if not marker in markers:
                print("Marker not found")
                return -1
            result[:, col] = self.Data[nLine, self.dataIdx[marker]: self.dataIdx[marker]+3]
        return result

In [4]:
class body:
    def __init__(self, file, markers, weights=None, iterations=3):
        # Computes the local body vecs for the markers
        self._markers = markers
        self._weights = np.ones((len(markers),))
        if (type(weights) == np.ndarray):
            self._weights = weights
        
        self._bodyVecs = file.getDataForMarkers(0, self._markers)        
        transformationMatrices = [None]*file.NumFrames

        for i in range(iterations):
            # Computing the transformation matrixes
            transformationMatrices = self.computeTransformMats(file)
            
            # Updating the local body vecs
            for j in range(len(self._markers)):
                localMarkerVecs = np.ones((4, file.NumFrames))
                for m in range(file.NumFrames):
                    # Inverse Transformation matrix
                    Tinv_t = np.identity(4)
                    Tinv_t[0:3, 0:3] = transformationMatrices[m][0:3, 0:3].T
                    Tinv_t[0:3,3] = -transformationMatrices[m][0:3, 3]

                    globalVec_t = np.ones((4,))
                    globalVec_t[0:3,] = file.getDataForMarkers(m, [self._markers[j]]).reshape(3,)
                    localMarkerVecs[:, m] = np.matmul(Tinv_t, globalVec_t)
                    
                T_t0 = transformationMatrices[0]
                self._bodyVecs[:, j] = np.matmul(T_t0, np.mean(localMarkerVecs, axis=1))[0:3]
        
    def computeTransformMats(self, file):
        transformationMatrices = [None]*file.NumFrames        
        for i in range(file.NumFrames):
            globalVecs = file.getDataForMarkers(i, self._markers)
            transformationMatrices[i] = TransformationMatrix(self._bodyVecs, globalVecs, self._weights)
        return transformationMatrices

In [5]:
def computeFJC(parent_transformMats, child_transformMats):    
    nRows = len(parent_transformMats)
    A = np.zeros((3*nRows, 6))
    b = np.zeros((3*nRows, 1))
    
    for i in range(nRows):
        parent_transformMat = parent_transformMats[i]
        child_transformMat = child_transformMats[i]

        R1 = parent_transformMat[0:3, 0:3]
        R2 = child_transformMat[0:3, 0:3]
        A[3*i:3*(i+1), :] = np.concatenate((np.identity(3), -np.matmul(R1.T, R2)), axis=1);
    
        child_parent_pos_global = child_transformMat[0:3,3] - parent_transformMat[0:3,3]
        child_parent_pos = np.matmul(R1.T,child_parent_pos_global)
        b[3*i:3*(i+1), 0] = child_parent_pos

    sHJC_lHJC = np.linalg.lstsq(A,b, rcond=-1)[0]

    # Joint Center in Parent fixed frame
    sHJC = np.concatenate((sHJC_lHJC[0:3,0], np.array([1])))

    # Joint Center in Child fixed frame
    lHJC = np.concatenate((sHJC_lHJC[3:,0], np.array([1])))
    
    return sHJC, lHJC

In [6]:
rHjcFileName = "RightCross1.csv"
#rAjfileName = "RAJC.csv"
lHjcFileName = "LeftCross1.csv"
#lAjfileName = "LAJC.csv"

rightArcFile = fileReader(rHjcFileName)
leftArcFile = fileReader(lHjcFileName)

print("Computing body vecs")
# Defines the local vecs
pelvisR = body(rightArcFile, ["RAsis", "LAsis", "LPsi"])
pelvisL = body(leftArcFile, ["RAsis", "LAsis", "RPsi"])
femurR = body(rightArcFile, ["RThigh1", "RThigh2", "RThigh3", "RKneeOut", "RKneeIn"], np.array([1, 1, 1, 0, 0]))
femurL = body(leftArcFile, ["LThigh1", "LThigh2", "LThigh3", "LKneeOut", "LKneeIn"], np.array([1, 1, 1, 0, 0]))

print("Computing joint centers")
sRHJC = computeFJC(pelvisR.computeTransformMats(rightArcFile), femurR.computeTransformMats(rightArcFile))[0]
sLHJC = computeFJC(pelvisL.computeTransformMats(leftArcFile), femurL.computeTransformMats(leftArcFile))[0]

Computing body vecs
Computing joint centers


In [7]:
tPoseFileName = "TPosePelvis2.csv"
file = fileReader(tPoseFileName)
nRows = file.NumFrames
#nRows = 1
out = np.zeros((nRows, 3*8))

pelvisR_transformMats = pelvisR.computeTransformMats(file)
pelvisL_transformMats = pelvisL.computeTransformMats(file)

for i in range(nRows):
    ## Right leg
    pelvis_transformMat_t = pelvisR_transformMats[i]
    rHJC_global = np.matmul(pelvis_transformMat_t, sRHJC)
    rKJC_global = np.mean(file.getDataForMarkers(i, ["RKneeOut", "RKneeIn"]), axis=1)
    
    ## Left leg
    pelvis_transformMat_t = pelvisL_transformMats[i]
    lHJC_global = np.matmul(pelvis_transformMat_t, sLHJC)
    lKJC_global = np.mean(file.getDataForMarkers(i, ["LKneeOut", "LKneeIn"]), axis=1)
    
    vin = lHJC_global-rHJC_global
    print(vin[2])
    
    # Derived markers
    midHJC_global = (rHJC_global+lHJC_global)/2
    midAsis = np.mean(file.getDataForMarkers(i, ["RAsis", "LAsis"]), axis=1)
    midPsis = np.mean(file.getDataForMarkers(i, ["RPsi", "LPsi"]), axis=1)
    midPelvis = (midAsis+midPsis)/2
    
    out[i,:] = np.concatenate((midAsis, midPsis, midPelvis,
                               rHJC_global[0:3], lHJC_global[0:3], midHJC_global[0:3],
                               rKJC_global[0:3], lKJC_global[0:3]), axis=0)

np.savetxt("vin.csv", out, delimiter=",")

-194.81469959795442
-194.8162902132029
-194.81775707359782
-194.81939342801752
-194.82090291037917
-194.82257825911785
-194.82413430996547
-194.82574665058058
-194.82718059041252
-194.82861295865115
-194.8298534193462
-194.83094098600384
-194.83185509279517
-194.83260150181096
-194.8334870074683
-194.83389296297872
-194.83442794462547
-194.83444983521792
-194.83446676118436
-194.8343862491917
-194.83426905469352
-194.83386143857803
-194.83360163521755
-194.83333690758016
-194.83327185738466
-194.8330971803096
-194.83305390247213
-194.8332550275532
-194.83327062246983
-194.83351693548514
-194.83376458138468
-194.83375473657284
-194.83375331592455
-194.83359867843015
-194.83343087763302
-194.83317410118002
-194.83301736732471
-194.83257157617714
-194.8323008391809
-194.83212485157048
-194.83209399097404
-194.83197168956417
-194.83198544980553
-194.8318567564835
-194.83193715395328
-194.8320628717962
-194.83225677928
-194.8322018913515
-194.83218443034178
-194.8323642435679
-194.832488400

In [8]:
midAsis = np.array([0.00696365,0.0103228,0.0])
misPsis = np.array([-0.153155,0.0426324,0.0])

midPelvis = np.mean((midAsis, misPsis), axis=0)

#R1Meta = np.array([0.174285,-0.0147377,-0.0298436])
#R5Meta = np.array([0.137032,-0.0147377,0.0412118])
#RmidMeta = np.mean((R1Meta, R5Meta), axis=0)

print(midPelvis)
#print(RmidMeta)

[-0.07309568  0.0264776   0.        ]
